<a href="https://colab.research.google.com/github/daystram/ml-playground/blob/master/01_gym_cartpole/01b_gym_cartpole_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01b Gym CartPole using DNN

In [0]:
!rm -r ml-playground > /dev/null 2>&1
!git clone https://github.com/daystram/ml-playground.git
!sh ml-playground/helper/setup.sh > /dev/null 2>&1

Cloning into 'ml-playground'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 116 (delta 66), reused 64 (delta 27), pack-reused 0
Receiving objects: 100% (116/116), 44.03 KiB | 2.32 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [0]:
import sys
sys.path.append('/content/ml-playground')

In [0]:
import gym

env = gym.make("CartPole-v1")
print(env.observation_space)
print(env.action_space)

Box(4,)
Discrete(2)


In [0]:
import numpy as np

acc_fitness = []
training_data = []

sample_count = 200000
steps = 1000
goal_tres = 128

for _ in range(sample_count):
  fitness = 0
  action_seq = []
  prev_observation = env.reset()
  
  for _ in range(steps):
    action = np.random.randint(0, 2)
    action_seq.append([prev_observation, action])
    prev_observation, reward, done, _ = env.step(action)
    fitness += reward
    if done: break
    
  if fitness >= goal_tres:
    acc_fitness.append(fitness)
    for seq in action_seq:
      if seq[1] == 1:
        action = [0, 1]
      else:
        action = [1, 0]
      training_data.append([seq[0], action])
env.close()
print("{} samples".format(len(training_data)))
print(np.array(acc_fitness))

1549 samples
[157. 153. 129. 128. 136. 137. 148. 131. 147. 151. 132.]


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

drop_rate = 0.3
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(4,)))
model.add(Dropout(drop_rate))
model.add(Dense(256, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(512, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(256, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(128, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(2, activation='relu'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
X = np.array([i[0] for i in training_data])
y = np.array([i[1] for i in training_data])

model.fit(X, y, epochs=20, batch_size=32)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
1549/1549 [==============================] - 2s 1ms/step - loss: 0.8642 - acc: 0.5352
Epoch 2/20
1549/1549 [==============================] - 0s 145us/step - loss: 0.6716 - acc: 0.5913
Epoch 3/20
1549/1549 [==============================] - 0s 153us/step - loss: 0.6639 - acc: 0.6068
Epoch 4/20
1549/1549 [==============================] - 0s 155us/step - loss: 0.6577 - acc: 0.6191
Epoch 5/20
1549/1549 [==============================] - 0s 154us/step - loss: 0.6555 - acc: 0.6198
Epoch 6/20
1549/1549 [==============================] - 0s 149us/step - loss: 0.6539 - acc: 0.6139
Epoch 7/20
1549/1549 [==============================] - 0s 148us/step - loss: 0.6472 - acc: 0.6139
Epoch 8/20
1549/1549 [==============================] - 0s 154us/step - loss: 0.6537 - acc: 0.6262
Epoch 9/20
1549/1549 [==============================] - 0s 147us/step - loss: 0.6491 - acc: 0.6288
Epoch 10/20
1549/1549 [==============================] - 0s 150

In [0]:
from helper import nbdisplay as disp

!rm -r video
env = disp.wrap_env(gym.make("CartPole-v1"))
observation = env.reset()
fitness = 0
print(np.array([observation]))
for _ in range(steps):
  action = np.argmax(model.predict(np.array([observation])))
  observation, reward, done, _ = env.step(action)
  fitness += reward
  if done: break

env.close()
disp.show_video()
print("Video can be viewed in Google Colab, use the link on top of the page to open")
print("------ Agent: Reward {}".format(fitness))  

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


[[ 0.02437529 -0.01411645  0.02295546 -0.00485035]]


Video can be viewed in Google Colab, use the link on top of the page to open
------ Agent: Reward 500.0


In [0]:
import pickle
import time

filename = '02-agent-{}.model'.format(time.strftime("%Y%m%d_%H%M%S"))
file = open(filename, 'wb')
pickle.dump(model, file=file)
file.close()